In [1]:
from edbo.plus.optimizer_botorch import EDBOplus
from edbo.edbo.feature_utils import reaction_space

C:\Anaconda3\envs\edbo\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### This is a tutorial that covers the basics for running EDBO+: from designing a combinatorial space to running the Bayesian Optimizer.

## 1. Creating a search scope using EDBO+.

##### To run EDBO+ we need to first create a reaction scope (search space) in a .csv format with all the possible combinations that you want to consider for our optimization. 
##### You can "manually" create a reaction scope using any spreadsheet editor (such as Excel, Libreoffice Calc, ...) but we have also created a tool to help you generating combinatorial search spaces. 
##### For instance, lets say that we want to consider 3 solvents ($\bf{THF}$, $\bf{Toluene}$, $\bf{DMSO}$), 4 temperatures ($\bf{-10}$, $\bf{0}$, $\bf{10}$, $\bf{25}$) and 3 concentration levels ($\bf{0.1}$, $\bf{0.2}$, $\bf{1.0}$). We can introduce these in the EDBO+ scope generator in a dictionary form as follows:

In [2]:
### 1_dataprocess 에서 저장한 dataset 파일을 불러와 BO를 진행. 적절한 변수를 이용해 필요한 모든 dataset을 불러옵니다. 

import os
import pandas as pd
from edbo.edbo.utils import Data


### 결과 폴더 지정. 실제 csv 파일이 있는 경로를 지정합니다. FILE_PREFIX는 추후 저장할 파일명에 사용됨
FOLDER = './result/'
FILE_PREFIX = 'my_optimization'

### 적절한 csv 파일명과 변수 명을 설정해주세요. 
sample = Data(pd.read_csv(os.path.join(FOLDER, 'sample.csv')))   
### 파일이 여러개일 경우 아래와 같은 포맷으로 파일 로드
#ligand = Data(pd.read_csv(os.path.join(FOLDER, 'ligand.csv')))   
#solvent = Data(pd.read_csv(os.path.join(FOLDER, 'solvent.csv')))   


# Use Data.drop method to drop descriptors containing some unwanted keywords
#for data in [azadicarbs, phosphines, solvents]:
#    data.drop(['file_name', 'entry', 'vibration', 'correlation', 'Rydberg', 
#               'correction', 'atom_number', 'E-M_angle', 'MEAN', 'MAXG', 
#               'STDEV'])

In [3]:
sample.data

,can,sample_E_boltz,sample_ES_root_dipole_boltz,sample_ES_root_electronic_spatial_extent_boltz,sample_ES_root_molar_volume_boltz,sample_E_scf_boltz,sample_E_thermal_correction_boltz,sample_E_zpe_boltz,sample_G_boltz,sample_G_thermal_correction_boltz,...,sample_electronegativity_std,sample_electronic_spatial_extent_std,sample_hardness_std,sample_homo_energy_std,sample_lumo_energy_std,sample_molar_mass_std,sample_molar_volume_std,sample_multiplicity_std,sample_number_of_atoms_std,sample_zero_point_correction_std
0,C(C1C(C(C(C(O1)O)O)O)O)O,-686.483139,3.655941,1945.71727,1424.188353,-686.687703,0.212099,-686.494718,-686.531304,0.163934,...,0.004775,112.662802,0.0038,0.006722,0.005414,0.0,107.480318,0.0,0.0,0.001122
1,c1ccc(F)cc1,-331.128965,1.302600,647.67130,945.523000,-331.226739,0.098046,-331.134091,-331.162907,0.064104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
### 위에서 읽어들인 DFT 파일을 이용해 Bayesian Optimization Scope 을 생성합니다. 
### 아래 components와 dft 설정에 위에서 load한 data를 기재해주세요. 

components = {'sample':'DFT',                             # DFT descriptors
              #'phosphine':'DFT',                                    # DFT descriptors
              #'solvent':'DFT',                                      # DFT descriptors
              'substrate_concentration':[0.05, 0.10],   # Discrete grid of concentrations
              'azadicarb_equiv':[1.1, 1.3, 1.5],          # Discrete grid of equiv.
              'phos_equiv':[1.5, 1.7, 1.9],               # Discrete grid of equiv.
              'temperature':[5, 15, 25]}                    # Discrete grid of temperatures

### Encodings - if not specified EDBO will automatically use OHE

encoding = {'substrate_concentration':'numeric',                    # Numerical encoding
            'azadicarb_equiv':'numeric',                            # Numerical encoding
            'phos_equiv':'numeric',                                 # Numerical encoding
            'temperature':'numeric'}                                # Numerical encoding

### External descriptor matrices override specified encoding

dft = { 'sample' : sample.data
     #   'azadicarboxylate':azadicarbs.data,                          # Unprocessed descriptor DataFrame
     #  'phosphine':phosphines.data,                                 # Unprocessed descriptor DataFrame
     #  'solvent':solvent.data}                                     # Unprocessed descriptor DataFrame
      }
df_scope = reaction_space(components, 
                    encoding=encoding,
                    descriptor_matrices=dft, clean=True, decorrelate=True)
df_scope.data.to_csv(os.path.join(FOLDER, f'{FILE_PREFIX}_scope.csv'))
df_scope.base_data.to_csv(os.path.join(FOLDER, f'{FILE_PREFIX}_scope_base.csv'))

In [5]:
df_scope.data

,sample_E_boltz,substrate_concentration,azadicarb_equiv,phos_equiv,temperature
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.5
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.5,0.0
4,0.0,0.0,0.0,0.5,0.5
...,...,...,...,...,...
103,1.0,1.0,1.0,0.5,0.5
104,1.0,1.0,1.0,0.5,1.0
105,1.0,1.0,1.0,1.0,0.0
106,1.0,1.0,1.0,1.0,0.5


##### We can always load/read the previously generated reaction scope using any spreadsheet editor but in this case we will use Pandas for that:

In [6]:
import pandas as pd
df_scope = pd.read_csv(os.path.join(FOLDER, f'{FILE_PREFIX}_scope.csv'), index_col=0)  # Load csv file.

##### Now we can check the number of combinations in the reaction scope:

In [8]:
n_combinations = len(df_scope)
print(f"Your reaction scope has {n_combinations} combinations.")

Your reaction scope has 108 combinations.


##### Of course, this is a very small reaction scope  as it is meant to be a toy model to demonstrate how EDBO+ works.

## 2. First steps, initializing EDBO+ (in absence of training data).

##### We are going to execute EDBO+ to suggest some initial samples. 
##### Since we have not collected any experimental data (observations) yet, EDBO+ will suggest a set of initial experiments based on an feature space sampling method, in this case the CVT sampling method (see:http://kmh-lanl.hansonhub.com/uncertainty/meetings/gunz03vgr.pdf).
##### In this example the $\bf{objective}$ is to maximize the reaction $\bf{yield}$ and $\bf{enantioselectivity}$ but at the same time we want to minimize the amount one of a given $\bf{side~product}$ in this reaction. We also need to introduce the name of the csv file containing our reaction scope (in our case this was $\bf{my\_optimization.csv}$). Now we can execute the algorithm using the $\bf{run}$ command in EDBOplus:

In [7]:
df_scope

,sample_E_boltz,substrate_concentration,azadicarb_equiv,phos_equiv,temperature
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.5
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.5,0.0
4,0.0,0.0,0.0,0.5,0.5
...,...,...,...,...,...
103,1.0,1.0,1.0,0.5,0.5
104,1.0,1.0,1.0,0.5,1.0
105,1.0,1.0,1.0,1.0,0.0
106,1.0,1.0,1.0,1.0,0.5


In [8]:
### objective 설정 필요. 실험에서 얻을 output 조건과 maximize or minimize할 것인지를 지정. 
EDBOplus().run(
    directory=FOLDER,
    filename=f'{FILE_PREFIX}_scope.csv',  # Previously generated scope.
    objectives=['yield', 'ee', 'side_product'],  # Objectives to be optimized.
    objective_mode=['max', 'max', 'min'],  # Maximize yield and ee but minimize side_product.
    batch=3,  # Number of experiments in parallel that we want to perform in this round.
    columns_features='all', # features to be included in the model.
    init_sampling_method='cvtsampling'  # initialization method.
)

Sampling type:  selection 


Number of unique samples returned by sampling algorithm: 3
Creating a priority list using random sampling: cvtsampling


,sample_E_boltz,substrate_concentration,azadicarb_equiv,phos_equiv,temperature,yield,ee,side_product,priority
42,0.0,1.0,0.5,1.0,0.0,PENDING,PENDING,PENDING,1
68,1.0,0.0,0.5,0.5,1.0,PENDING,PENDING,PENDING,1
36,0.0,1.0,0.5,0.0,0.0,PENDING,PENDING,PENDING,1
78,1.0,0.0,1.0,1.0,0.0,PENDING,PENDING,PENDING,0
77,1.0,0.0,1.0,0.5,1.0,PENDING,PENDING,PENDING,0
...,...,...,...,...,...,...,...,...,...
32,0.0,1.0,0.0,0.5,1.0,PENDING,PENDING,PENDING,0
31,0.0,1.0,0.0,0.5,0.5,PENDING,PENDING,PENDING,0
30,0.0,1.0,0.0,0.5,0.0,PENDING,PENDING,PENDING,0
29,0.0,1.0,0.0,0.0,1.0,PENDING,PENDING,PENDING,0


##### EDBO+ has created a column for each objective and added $\bf{PENDING}$ values to all of them so you can track the experiments that you have been collecting during the optimization campaign.
##### We can also see that EDBO+ has created a new $\bf{priority}$ column. This column is used to distinguish between high and low priority samples. The top entries (with $\bf{priority=1}$) highlight the next suggested samples.

##### We can check now the first 5 experiments in the scope by reading the $\bf{my\_optimization.csv}$ file:

In [9]:
df_edbo = pd.read_csv(os.path.join(FOLDER, f'{FILE_PREFIX}_scope.csv'), index_col=0)
df_edbo.head(5)

,sample_E_boltz,substrate_concentration,azadicarb_equiv,phos_equiv,temperature,yield,ee,side_product,priority
42,0.0,1.0,0.5,1.0,0.0,PENDING,PENDING,PENDING,1
68,1.0,0.0,0.5,0.5,1.0,PENDING,PENDING,PENDING,1
36,0.0,1.0,0.5,0.0,0.0,PENDING,PENDING,PENDING,1
78,1.0,0.0,1.0,1.0,0.0,PENDING,PENDING,PENDING,0
77,1.0,0.0,1.0,0.5,1.0,PENDING,PENDING,PENDING,0


In [14]:
df_edbo.head().index.tolist()   ## 확인하고자 하는 index 정보 확인

[42, 68, 36, 78, 77]

## Base condition 확인
df_scope을 생성하며 저장한 scope_base.csv 파일을 이용하여 해당 index의 내용을 확인합니다. 

In [54]:
df_base_scope = pd.read_csv(os.path.join(FOLDER, f'{FILE_PREFIX}_scope_base.csv'), index_col=0)
df_base_scope.head()

,can_index,sample_E_boltz,substrate_concentration_index,substrate_concentration,azadicarb_equiv_index,azadicarb_equiv,phos_equiv_index,phos_equiv,temperature_index,temperature
0,C(C1C(C(C(C(O1)O)O)O)O)O,-686.483139,0.05,0.05,1.1,1.1,1.5,1.5,5,5
1,C(C1C(C(C(C(O1)O)O)O)O)O,-686.483139,0.05,0.05,1.1,1.1,1.5,1.5,15,15
2,C(C1C(C(C(C(O1)O)O)O)O)O,-686.483139,0.05,0.05,1.1,1.1,1.5,1.5,25,25
3,C(C1C(C(C(C(O1)O)O)O)O)O,-686.483139,0.05,0.05,1.1,1.1,1.7,1.7,5,5
4,C(C1C(C(C(C(O1)O)O)O)O)O,-686.483139,0.05,0.05,1.1,1.1,1.7,1.7,15,15


In [16]:
df_base_scope.loc[df_edbo.head().index.tolist()]

,can_index,sample_E_boltz,substrate_concentration_index,substrate_concentration,azadicarb_equiv_index,azadicarb_equiv,phos_equiv_index,phos_equiv,temperature_index,temperature
42,C(C1C(C(C(C(O1)O)O)O)O)O,-686.483139,0.10,0.10,1.3,1.3,1.9,1.9,5,5
68,c1ccc(F)cc1,-331.128965,0.05,0.05,1.3,1.3,1.7,1.7,25,25
36,C(C1C(C(C(C(O1)O)O)O)O)O,-686.483139,0.10,0.10,1.3,1.3,1.5,1.5,5,5
78,c1ccc(F)cc1,-331.128965,0.05,0.05,1.5,1.5,1.9,1.9,5,5
77,c1ccc(F)cc1,-331.128965,0.05,0.05,1.5,1.5,1.7,1.7,25,25


## 3. Adding training data in EDBO+.

##### Note: We will use Python and Pandas to add new training data in this example. But you can always edit and add new data into the '.csv' file using any spreedsheet editor (such as Excel, Libreoffice Calc, ...) if that's more convinient for you.

##### Let's open again the $\bf{my\_optimization.csv}$ file we generated before:

In [34]:
## 아래와 같이 코드 상에서 값을 지정하거나 혹은 csv 파일에 직접 값을 기재하여 사용합니다. 

df_edbo = pd.read_csv(os.path.join(FOLDER, f'{FILE_PREFIX}_scope.csv'), index_col=0)
df_edbo.head(5)

,sample_E_boltz,substrate_concentration,azadicarb_equiv,phos_equiv,temperature,yield,ee,side_product,priority
42,0.0,1.0,0.5,1.0,0.0,PENDING,PENDING,PENDING,1
68,1.0,0.0,0.5,0.5,1.0,PENDING,PENDING,PENDING,1
36,0.0,1.0,0.5,0.0,0.0,PENDING,PENDING,PENDING,1
78,1.0,0.0,1.0,1.0,0.0,PENDING,PENDING,PENDING,0
77,1.0,0.0,1.0,0.5,1.0,PENDING,PENDING,PENDING,0


##### We can fill the first out entry in the previous dataframe with the "observed" values using Pandas:

In [44]:
df_edbo.iloc[0, df_edbo.columns.get_loc("yield")] = 20.5
df_edbo.iloc[0, df_edbo.columns.get_loc("ee")] = 40
df_edbo.iloc[0, df_edbo.columns.get_loc("side_product")]= 0.1

##### We can check that we have filled out the first entry with our "observed data":

In [45]:
df_edbo.head(5)

,sample_E_boltz,substrate_concentration,azadicarb_equiv,phos_equiv,temperature,yield,ee,side_product,priority
42,0.0,1.0,0.5,1.0,0.0,20.5,40,0.1,1
68,1.0,0.0,0.5,0.5,1.0,PENDING,PENDING,PENDING,1
36,0.0,1.0,0.5,0.0,0.0,PENDING,PENDING,PENDING,1
78,1.0,0.0,1.0,1.0,0.0,PENDING,PENDING,PENDING,0
77,1.0,0.0,1.0,0.5,1.0,PENDING,PENDING,PENDING,0


##### We can also fill out the second entry with their corresponding "observations":

In [46]:
df_edbo.iloc[1, df_edbo.columns.get_loc("yield")] = 50
df_edbo.iloc[1, df_edbo.columns.get_loc("ee")] = 10
df_edbo.iloc[1, df_edbo.columns.get_loc("side_product")]= 0.2

In [47]:
df_edbo.head(5)

,sample_E_boltz,substrate_concentration,azadicarb_equiv,phos_equiv,temperature,yield,ee,side_product,priority
42,0.0,1.0,0.5,1.0,0.0,20.5,40,0.1,1
68,1.0,0.0,0.5,0.5,1.0,50,10,0.2,1
36,0.0,1.0,0.5,0.0,0.0,PENDING,PENDING,PENDING,1
78,1.0,0.0,1.0,1.0,0.0,PENDING,PENDING,PENDING,0
77,1.0,0.0,1.0,0.5,1.0,PENDING,PENDING,PENDING,0


##### Now we can save our dataset as $\bf{my\_optimization\_round0.csv}$:

In [48]:
df_edbo.to_csv(os.path.join(FOLDER, f'{FILE_PREFIX}_round0.csv'))

## 4. Running EDBO+ with training data.

##### First let's check our previous data (which include some $\bf{yield}$, $\bf{ee}$ and $\bf{side\_product}$ observations, which will be used to train the model):

In [49]:
df_edbo_round0 = pd.read_csv(os.path.join(FOLDER, f'{FILE_PREFIX}_round0.csv'), index_col=0)
df_edbo_round0.head(5)

,sample_E_boltz,substrate_concentration,azadicarb_equiv,phos_equiv,temperature,yield,ee,side_product,priority
42,0.0,1.0,0.5,1.0,0.0,20.5,40,0.1,1
68,1.0,0.0,0.5,0.5,1.0,50,10,0.2,1
36,0.0,1.0,0.5,0.0,0.0,PENDING,PENDING,PENDING,1
78,1.0,0.0,1.0,1.0,0.0,PENDING,PENDING,PENDING,0
77,1.0,0.0,1.0,0.5,1.0,PENDING,PENDING,PENDING,0


##### Now that we have introduced some "observations" in our $\bf{my\_optimization\_round0.csv}$ file, we can execute EDBO+ to suggest samples using these "observations" as training data.

In [50]:
EDBOplus().run(
    directory=FOLDER,
    filename=f'{FILE_PREFIX}_round0.csv',  # Previous scope (including observations).
    objectives=['yield', 'ee', 'side_product'],  # Objectives to be optimized.
    objective_mode=['max', 'max', 'min'],  # Maximize yield and ee but minimize side_product.
    batch=3,  # Number of experiments in parallel that we want to perform in this round.
    columns_features='all', # features to be included in the model.
    init_sampling_method='cvtsampling'  # initialization method.
)

Using EHVI acquisition function.
Using hyperparameters optimized for continuous variables.
Using hyperparameters optimized for continuous variables.
Using hyperparameters optimized for continuous variables.
Number of QMC samples using SobolQMCNormalSampler sampler: 512
Acquisition function optimized.
Predictions obtained and expected improvement obtained.


,sample_E_boltz,substrate_concentration,azadicarb_equiv,phos_equiv,temperature,yield,ee,side_product,priority
52,0.0,1.0,1.0,1.0,0.5,PENDING,PENDING,PENDING,1.0
27,0.0,1.0,0.0,0.0,0.0,PENDING,PENDING,PENDING,1.0
8,0.0,0.0,0.0,1.0,1.0,PENDING,PENDING,PENDING,1.0
107,1.0,1.0,1.0,1.0,1.0,PENDING,PENDING,PENDING,0.0
106,1.0,1.0,1.0,1.0,0.5,PENDING,PENDING,PENDING,0.0
...,...,...,...,...,...,...,...,...,...
2,0.0,0.0,0.0,0.0,1.0,PENDING,PENDING,PENDING,0.0
1,0.0,0.0,0.0,0.0,0.5,PENDING,PENDING,PENDING,0.0
0,0.0,0.0,0.0,0.0,0.0,PENDING,PENDING,PENDING,0.0
82,1.0,1.0,0.0,0.0,0.5,PENDING,PENDING,PENDING,-1.0


##### Again the samples suggested by EDBO+ have $\bf{priority = +1}$. In addition, we asign $\bf{priority = -1}$ to the experiments that we have already run (these are at the bottom of the dataset).

## Extra: Accessing the model predictions.

##### Each time that EDBO+ is executed with training data it will generate a .csv file with the predictions for the entire scope (including the 'untested' samples).
##### In the previous example, by running EDBO+ with training data, we generated two files: $\bf{my\_optimization\_round0.csv}$ and a second file with the predictions $\bf{pred\_my\_optimization\_round0.csv}$.
##### Let's have a look to the predictions file using Pandas:

In [52]:
df_predictions_round0 = pd.read_csv('./result/pred_my_optimization_round0.csv', index_col=0)
df_predictions_round0

,sample_E_boltz,substrate_concentration,azadicarb_equiv,phos_equiv,temperature,yield,ee,side_product,priority,yield_predicted_mean,yield_predicted_variance,yield_expected_improvement,ee_predicted_mean,ee_predicted_variance,ee_expected_improvement,side_product_predicted_mean,side_product_predicted_variance,side_product_expected_improvement
52,0.0,1.0,1.0,1.0,0.5,PENDING,PENDING,PENDING,1.0,32.639887,21.530197,9.876019,27.654352,21.895115,11.986662,0.141152,0.072984,0.244149
27,0.0,1.0,0.0,0.0,0.0,PENDING,PENDING,PENDING,1.0,35.250000,32.734962,19.403792,25.000000,33.289792,19.732670,0.150000,0.110966,0.264449
8,0.0,0.0,0.0,1.0,1.0,PENDING,PENDING,PENDING,1.0,35.250000,23.874847,12.576066,25.000000,24.279506,12.789220,0.150000,0.080932,0.254283
107,1.0,1.0,1.0,1.0,1.0,PENDING,PENDING,PENDING,0.0,35.250000,23.874847,12.576066,25.000000,24.279506,12.789220,0.150000,0.080932,0.254283
106,1.0,1.0,1.0,1.0,0.5,PENDING,PENDING,PENDING,0.0,34.382809,22.532399,11.238546,25.881889,22.914304,12.084671,0.147060,0.076381,0.250469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,0.0,0.0,0.0,0.0,1.0,PENDING,PENDING,PENDING,0.0,38.323997,31.220868,19.506888,21.873901,31.750035,17.294945,0.160420,0.105833,0.271575
1,0.0,0.0,0.0,0.0,0.5,PENDING,PENDING,PENDING,0.0,37.581796,30.832825,18.889086,22.628682,31.355415,17.286101,0.157904,0.104518,0.268842
0,0.0,0.0,0.0,0.0,0.0,PENDING,PENDING,PENDING,0.0,36.782693,32.358740,19.746471,23.441329,32.907193,18.803468,0.155196,0.109691,0.268460
82,1.0,1.0,0.0,0.0,0.5,PENDING,PENDING,PENDING,-1.0,37.581796,30.832825,18.889086,22.628682,31.355415,17.286101,0.157904,0.104518,0.268842


In [53]:
df_predictions_round0 = pd.read_csv('./result/pred_my_optimization_round0.csv', index_col=0)
df_predictions_round0.style.background_gradient(subset=['priority'], cmap='plasma')

,sample_E_boltz,substrate_concentration,azadicarb_equiv,phos_equiv,temperature,yield,ee,side_product,priority,yield_predicted_mean,yield_predicted_variance,yield_expected_improvement,ee_predicted_mean,ee_predicted_variance,ee_expected_improvement,side_product_predicted_mean,side_product_predicted_variance,side_product_expected_improvement
52,0.000000,1.000000,1.000000,1.000000,0.500000,PENDING,PENDING,PENDING,1.000000,32.639887,21.530197,9.876019,27.654352,21.895115,11.986662,0.141152,0.072984,0.244149
27,0.000000,1.000000,0.000000,0.000000,0.000000,PENDING,PENDING,PENDING,1.000000,35.250000,32.734962,19.403792,25.000000,33.289792,19.732670,0.150000,0.110966,0.264449
8,0.000000,0.000000,0.000000,1.000000,1.000000,PENDING,PENDING,PENDING,1.000000,35.250000,23.874847,12.576066,25.000000,24.279506,12.789220,0.150000,0.080932,0.254283
107,1.000000,1.000000,1.000000,1.000000,1.000000,PENDING,PENDING,PENDING,0.000000,35.250000,23.874847,12.576066,25.000000,24.279506,12.789220,0.150000,0.080932,0.254283
106,1.000000,1.000000,1.000000,1.000000,0.500000,PENDING,PENDING,PENDING,0.000000,34.382809,22.532399,11.238546,25.881889,22.914304,12.084671,0.147060,0.076381,0.250469
105,1.000000,1.000000,1.000000,1.000000,0.000000,PENDING,PENDING,PENDING,0.000000,33.539399,23.393978,11.578772,26.739594,23.790486,13.084282,0.144201,0.079302,0.248445
104,1.000000,1.000000,1.000000,0.500000,1.000000,PENDING,PENDING,PENDING,0.000000,36.960601,23.393978,12.866210,23.260406,23.790486,11.775022,0.155799,0.079302,0.259375
103,1.000000,1.000000,1.000000,0.500000,0.500000,PENDING,PENDING,PENDING,0.000000,36.117191,22.532399,11.883260,24.118111,22.914304,11.429030,0.152940,0.076381,0.256049
102,1.000000,1.000000,1.000000,0.500000,0.000000,PENDING,PENDING,PENDING,0.000000,35.250000,23.874847,12.576066,25.000000,24.279506,12.789220,0.150000,0.080932,0.254283
101,1.000000,1.000000,1.000000,0.000000,1.000000,PENDING,PENDING,PENDING,0.000000,38.323997,31.220868,19.506888,21.873901,31.750035,17.294945,0.160420,0.105833,0.271575


##### In the previous dataset we can access the model predictions ($\bf{predicted\_mean}$ and $\bf{predicted\_variance}$ columns) but also to the expected improvement values (in the $\bf{expected\_improvement}$ columns) for each objective ($\bf{yield}$, $\bf{ee}$ and $\bf{side\_product}$.).